# Composition de musique note par note

Voici notre projet de 2A

## Introduction

Voici une introduction

Lorsque il y a ambiguité, entre le nom d'une variable et un nom communs, le nom de la variable est notés entre --, par exemple, 'les notes de la liste -notes-' ici le premier mot notes désigne les notes au sens commun (Do, Ré, Mi ...) tandis que -notes- désigne le nom de la variable python

### Imports

```python
import numpy as np
from __future__ import print_function
import tensorflow
from utils import *
import IPython
import sys
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
```

## Description utils
utils est l'ensemble des fonctions permettant la relation entre les fichiers musiques en midi (via music21) et des matrices numpy permettant l'abstraction des dîtes musiques afin de faire passer l'analogie des musiques (matrices) au sein du réseau de neurones. (Bijection entre musique et matrices)


### Import de utils
```python
from keras.utils import to_categorical
from music21 import *
import numpy as np
```

### get_notes(path_to_midi)

#### argument(s) :
- path_to_midi est une chaine de caractère representant le chemin de fichier midi

#### retour :
get_notes(path_to_midi) renvoie une liste (-notes-) dont les éléments sont un à un les notes de la musique midi (ici notes et accords sont synonymes excepté que les notes sont représentées par un caractère et les accords par une suite de chiffres relié par des points

```python
def get_notes(path_to_midi):
    notes = []

    midi = converter.parse(path_to_midi)

    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            # for n in element.pitches:
            #   print(str(n))
            notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

```

### generate_vocab(notes)

#### argument(s) :
- notes est une liste dont les éléments sont des chaînes de caractères (lettre pour les notes, suite de chiffres séparés par des points pour les accord)

#### retour :
 generate_vocab(notes) renvoie l'ensemble du vocabulaire, c'est à dire la liste des éléments distincts de notes

```python
def generate_vocab(notes):
    """Generate vocabulary based on the input notes"""
    return np.unique(np.array(notes))
```

### generate_X_Y_from_one_music(note_to_index, notes, Tx, m)

#### argument(s) :
- note_to_index est un dictionnaire reliant les notes aux indexes par rapport au vocabulaire 
- notes est une liste dont les éléments sont des chaînes de caractères (lettre pour les notes, suite de chiffres séparés par des points pour les accord)
- Tx est la longueur d'un training examples
- m est le nombre de séquences souhaités pour une musique donnés

#### retour :
generate_X_Y_from_one_music(name_to_index, notes, Tx, m) renvoie X et Y où X et Y sont des tenseurs à 3 dimensions comprenant l'ensemble des 'trainings examples', qui pour chaque 'training example' (ici ce nombre correspond au nombres de séquences de Tx notes, cette séquence est choisie aléatoirement dans la suite de notes -notes-) il y a une suite de Tx notes

```python
def generate_X_Y_from_one_music(name_to_index, notes, Tx, m):
    """Generate vectors X and Y for training where X[i+1]=Y[i]"""
    N_values = len(note_to_index)
    np.random.seed(0)
    X = np.zeros((m, Tx, N_values), dtype=np.bool)
    Y = np.zeros((m, Tx, N_values), dtype=np.bool)
    for i in range(m):
        random_idx = np.random.choice(len(notes) - Tx)
        notes_data = notes[random_idx:(random_idx + Tx)]
        for j in range(Tx):
            idx = note_to_index[notes_data[j]]
            if j != 0:
                X[i, j, idx] = 1
                Y[i, j - 1, idx] = 1

    Y = np.swapaxes(Y, 0, 1)
    Y = Y.tolist()
    return np.asarray(X), np.asarray(Y)
```

### generate_midi_file(outputName, notes)

#### argument(s) :
- outputName est une chaîne de caractères (se terminant par .mid) désignant le nom du fichier midi généré  
- notes est une liste dont les éléments sont des chaînes de caractères (lettre pour les notes, suite de chiffres séparés par des points pour les accord)

#### retour :
generate_midi_file(outputName, notes) écrit le fichier midi 'outputName' en placant à la suites les notes de -notes-

```python
def generate_midi_file(outputName, notes):
    sheet = stream.Stream()
    for x in notes:
        if RepresentsInt(x[0]):
            ch = x.split(".")
            sheet.append(chord.Chord([int(k) for k in ch], quarterLength=0.25))
        else:
            sheet.append(note.Note(x, quarterLength=0.25))
    mf = midi.translate.streamToMidiFile(sheet)
    mf.open(outputName, "wb")
    mf.write()
    mf.close()
```



